In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
df = pd.read_csv("sales_pos.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


### Q1.

In [4]:
ser_g = df.groupby("prod")["purchase"].sum()
ser_g.max()

27995166

In [5]:
ser_g[ser_g == ser_g.max()]

prod
P00025442    27995166
Name: purchase, dtype: int64

In [6]:
top_prod = ser_g.idxmax()
top_prod

'P00025442'

In [7]:
df_q1 = df.loc[df["prod"] == top_prod, ]
df_q1["job"].value_counts().idxmax()

4

### Q2.

In [8]:
df_u1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]].reset_index(drop = True)
df_u1 = df_u1.fillna(0)
df_u1.head(2)

,prod_cat1,prod_cat2,prod_cat3
0,3,0.0,0.0
1,1,6.0,14.0


In [9]:
df_u1.shape

(35, 3)

In [10]:
# df_u1.sort_values(df_u1.columns.to_list())

In [11]:
df_u1.drop_duplicates().shape

(21, 3)

In [12]:
df_u1["prod_cat1"] = df_u1["prod_cat1"].astype("str")
df_u1["prod_cat2"] = df_u1["prod_cat2"].astype("int").astype("str") # .astype("int") 는 굳이 안해도 됨.
df_u1["prod_cat3"] = df_u1["prod_cat3"].astype("int").astype("str")
df_u1["prod_cat"] = df_u1["prod_cat1"] + "-" + df_u1["prod_cat2"] + "-" + df_u1["prod_cat3"]

In [13]:
df_u1["prod_cat"].nunique()

21

In [14]:
df_q2 = df.loc[df["age_group"] == "26-35", ["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2 = df_q2.reset_index(drop = True)
df_q2 = df_q2.fillna(0)
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str") # .astype("int") 는 굳이 안해도 됨.
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat" ] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,1,2,0,1-2-0
1,5,1,8,0,0,8-0-0


In [15]:
df_g = df_q2.groupby(["user", "marital"])["prod_cat"].nunique().reset_index()
df_g.head()

,user,marital,prod_cat
0,3,0,18
1,5,1,43
2,8,1,32
3,9,0,31
4,11,0,34


In [16]:
stat_m0 = df_g.loc[df_g["marital"] == 0, "prod_cat"].mean()
stat_m1 = df_g.loc[df_g["marital"] == 1, "prod_cat"].mean()
stat_m0, stat_m1

(41.66318327974277, 41.79233621755253)

In [17]:
round(abs(stat_m0 - stat_m1), 2)

0.13

### Q3.

In [18]:
df["user"].nunique()

5891

In [19]:
df_user = df.iloc[:, [0, 2, 3, 4, 5, 6]].drop_duplicates()
df_user.head(1)

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0


In [20]:
df_g = df.groupby("user")[["prod", "purchase"]].agg({"prod": "nunique",
                                                     "purchase": "sum"}).reset_index()
df_g.head(1)

,user,prod,purchase
0,1,35,334093


In [21]:
df_join = pd.merge(df_user, df_g, on = "user", how = "inner")
df_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472


In [22]:
ls_col_g = ["user", "gender",	"age_group", "job", "city", "marital"]
df_join = df.groupby(ls_col_g)[["prod", "purchase"]].agg({"prod": "nunique",
                                                          "purchase": "sum"}).reset_index()
df_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472


In [23]:
df_join["gender"] = df_join["gender"].replace({"M": 1, "F": 0})

In [24]:
ser_repl = pd.Series(range(7), index = df_join["age_group"].sort_values().unique())
ser_repl

0-17     0
18-25    1
26-35    2
36-45    3
46-50    4
51-55    5
55+      6
dtype: int64

In [25]:
df_join["age_group"] = df_join["age_group"].replace(ser_repl)

In [26]:
df_join.head()

,user,gender,age_group,job,city,marital,prod,purchase
0,1,0,0,10,A,0,35,334093
1,2,1,6,16,C,0,77,810472
2,3,1,2,15,A,0,29,341635
3,4,1,4,7,B,1,14,206468
4,5,1,2,20,A,1,106,821001


In [27]:
# df_join_dum = pd.get_dummies(df_join, columns = ["job", "city"]) # 시험버전
df_join_dum = pd.get_dummies(df_join, columns = ["job", "city"], dtype = "int")
df_join_dum = df_join_dum.drop(columns = "user")
df_join_dum.head(2)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,0,0,0,35,334093,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,6,0,77,810472,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [30]:
arr_join_nor = MinMaxScaler().fit_transform(df_join_dum)
arr_join_nor[:1, ]

array([[0.        , 0.        , 0.        , 0.02843137, 0.02739807,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ]])

In [78]:
model_kmeans = KMeans(n_clusters = 7, random_state = 123)
model_kmeans.fit(arr_join_nor)
round(silhouette_score(arr_join_nor, labels = model_kmeans.labels_), 2)

0.18

In [81]:
ser_c = pd.Series(model_kmeans.labels_)
ser_c.value_counts() # 각 군집별 크기

1    1204
0    1187
2    1133
5    1045
3     503
4     433
6     386
Name: count, dtype: int64

In [89]:
df_cl = pd.DataFrame(model_kmeans.cluster_centers_, 
                     columns = df_join_dum.columns)
df_cl

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,6.933446e-01,0.560095,1.000000e+00,0.048312,0.046656,1.280539e-01,1.162595e-01,4.128054e-02,4.043808e-02,0.070767,...,0.072452,2.948610e-02,6.655434e-02,1.002527e-01,1.937658e-02,6.739680e-03,4.212300e-02,3.330669e-16,-5.551115e-17,1.000000e+00
1,1.000000e+00,0.413760,4.053156e-01,0.135295,0.123965,1.129568e-01,6.810631e-02,4.401993e-02,1.744186e-02,0.127907,...,0.048173,2.990033e-02,4.069767e-02,1.013289e-01,9.966777e-03,9.966777e-03,6.146179e-02,3.330669e-16,1.000000e+00,-3.330669e-16
2,1.000000e+00,0.364225,-5.551115e-16,0.050233,0.049510,1.350397e-01,7.060900e-02,3.971756e-02,1.853486e-02,0.150927,...,0.045013,2.471315e-02,4.413063e-02,1.385702e-01,2.118270e-02,2.294793e-02,3.795234e-02,3.053113e-16,-1.665335e-16,1.000000e+00
3,3.330669e-16,0.416832,4.274354e-01,0.106767,0.089099,1.332008e-01,1.153082e-01,5.367793e-02,5.168986e-02,0.163022,...,0.049702,1.391650e-02,2.385686e-02,2.783300e-02,1.040834e-17,1.192843e-02,6.361829e-02,-2.775558e-17,1.000000e+00,-2.220446e-16
4,2.220446e-16,0.379523,5.551115e-17,0.041165,0.036954,1.177829e-01,1.501155e-01,3.233256e-02,4.618938e-02,0.177829,...,0.048499,2.078522e-02,2.309469e-02,3.926097e-02,2.309469e-03,1.154734e-02,3.464203e-02,-2.775558e-17,-1.110223e-16,1.000000e+00
5,7.177033e-01,0.399522,3.760766e-01,0.132705,0.115641,1.234450e-01,8.995215e-02,6.507177e-02,3.253589e-02,0.164593,...,0.050718,2.392344e-02,3.349282e-02,5.933014e-02,6.698565e-03,1.339713e-02,5.645933e-02,1.000000e+00,-1.665335e-16,-1.110223e-16
6,8.160622e-01,0.532815,4.948187e-01,0.044740,0.044455,1.387779e-17,-2.775558e-17,6.938894e-18,1.040834e-17,0.000000,...,0.000000,-1.387779e-17,1.387779e-17,-2.775558e-17,5.204170e-18,-1.734723e-18,6.938894e-18,-2.775558e-17,-5.551115e-17,1.000000e+00
